# Predictor y productor de música con IA

##Maxime Vilcocq Parra, A01710550
##Módulo 2 Implementación de un modelo de deep learning. (Portafolio Implementación)

Problema: A pesar de que siempre me ha gustado la música, no tengo ni la más remota idea de cómo funciona. Por ello, núnca podría crear una canción por mi cuenta.

En este trabajo, pretendo utilizar deeplearning para entrenar a un compositor de IA.

#Extracción

Dependencias

In [1]:

!apt-get install -y fluidsynth
!pip install -q music21 pretty_midi midi2audio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0
  libinput-bin libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libx

music21: librería para analizar y manipular partituras MIDI.

pretty_midi + midi2audio: permiten convertir un .mid a WAV y escucharlo.

fluidsynth: sintetizador que reproduce el audio directamente en Colab.

In [2]:
import os
import json
import urllib.request
import numpy as np
from pathlib import Path
from music21 import converter, instrument, note, chord, corpus

In [3]:
# Carpeta base del proyecto
BASE_DIR = Path("/content/melody_project")
DATA_DIR = BASE_DIR / "data"

midi_clasico  = DATA_DIR / "midi_clasico"
midi_videojuegos = DATA_DIR / "midi_videojuegos"
midi_otro   = DATA_DIR / "midi_otro" #de momento no lo ocupo

# Crear las carpetas
for folder in [midi_clasico, midi_videojuegos, midi_otro]:
    folder.mkdir(parents=True, exist_ok=True)

print("Carpetas creadas:")
print(midi_clasico)
print(midi_videojuegos)
print(midi_otro)


Carpetas creadas:
/content/melody_project/data/midi_clasico
/content/melody_project/data/midi_videojuegos
/content/melody_project/data/midi_otro


##Descarga de música clásica con music21

In [4]:
# Copiamos unas piezas de Bach al directorio de música clásica
corales = corpus.getComposer('bach')[:5]
print(f"Se usarán {len(corales)} corales de Bach")

for idx, pieza_path in enumerate(corales):
    midi_stream = converter.parse(pieza_path)
    out_path = midi_clasico / f"bach_{idx}.mid"
    midi_stream.write('midi', fp=out_path)

print("\nArchivos de música clásica listos en:")
!ls -lh {midi_clasico}


Se usarán 5 corales de Bach

Archivos de música clásica listos en:
total 24K
-rw-r--r-- 1 root root 4.6K Nov  5 22:36 bach_0.mid
-rw-r--r-- 1 root root 2.1K Nov  5 22:36 bach_1.mid
-rw-r--r-- 1 root root 2.0K Nov  5 22:36 bach_2.mid
-rw-r--r-- 1 root root 2.2K Nov  5 22:36 bach_3.mid
-rw-r--r-- 1 root root 3.1K Nov  5 22:36 bach_4.mid


##Descarga de música externa manual

use la página de https://bitmidi.com/

In [8]:
import requests

game_urls = {
    "super_mario.mid": "https://bitmidi.com/uploads/98137.mid",
    "wii.mid": "https://bitmidi.com/uploads/110325.mid",
    "zelda1.mid": "https://bitmidi.com/uploads/101446.mid",
    "zelda2.mid": "https://bitmidi.com/uploads/112951.mid",
    "zelda3.mid": "https://bitmidi.com/uploads/112955.mid",
}

headers = {"User-Agent": "Mozilla/5.0"}

for fname, url in game_urls.items():
    out_path = midi_videojuegos / fname
    print(f"Descargando {fname} desde {url}")
    try:
        r = requests.get(url, headers=headers, timeout=20)
        r.raise_for_status()  # genera excepción si hay error
        with open(out_path, "wb") as f:
            f.write(r.content)
        print(f"Guardado: {fname}")
    except Exception as e:
        print(f"Error descargando {fname}: {e}")

print("\nArchivos de videojuegos listos en:")
!ls -lh {midi_videojuegos}


Descargando super_mario.mid desde https://bitmidi.com/uploads/98137.mid
Guardado: super_mario.mid
Descargando wii.mid desde https://bitmidi.com/uploads/110325.mid
Guardado: wii.mid
Descargando zelda1.mid desde https://bitmidi.com/uploads/101446.mid
Guardado: zelda1.mid
Descargando zelda2.mid desde https://bitmidi.com/uploads/112951.mid
Guardado: zelda2.mid
Descargando zelda3.mid desde https://bitmidi.com/uploads/112955.mid
Guardado: zelda3.mid

Archivos de videojuegos listos en:
total 76K
-rw-r--r-- 1 root root  48K Nov  5 22:37 super_mario.mid
-rw-r--r-- 1 root root 5.0K Nov  5 22:37 wii.mid
-rw-r--r-- 1 root root 4.5K Nov  5 22:37 zelda1.mid
-rw-r--r-- 1 root root  515 Nov  5 22:37 zelda2.mid
-rw-r--r-- 1 root root 4.7K Nov  5 22:37 zelda3.mid


In [9]:
!ls -lh {midi_clasico}
!ls -lh {midi_videojuegos}


total 24K
-rw-r--r-- 1 root root 4.6K Nov  5 22:36 bach_0.mid
-rw-r--r-- 1 root root 2.1K Nov  5 22:36 bach_1.mid
-rw-r--r-- 1 root root 2.0K Nov  5 22:36 bach_2.mid
-rw-r--r-- 1 root root 2.2K Nov  5 22:36 bach_3.mid
-rw-r--r-- 1 root root 3.1K Nov  5 22:36 bach_4.mid
total 76K
-rw-r--r-- 1 root root  48K Nov  5 22:37 super_mario.mid
-rw-r--r-- 1 root root 5.0K Nov  5 22:37 wii.mid
-rw-r--r-- 1 root root 4.5K Nov  5 22:37 zelda1.mid
-rw-r--r-- 1 root root  515 Nov  5 22:37 zelda2.mid
-rw-r--r-- 1 root root 4.7K Nov  5 22:37 zelda3.mid


Extraer las notas

In [10]:
#Extrae notas y acordes de todos los archivos MIDI en una carpeta
def extract_notes(midi_folder):

    notes = []
    for file in os.listdir(midi_folder):
        if not file.endswith(".mid"):
            continue
        fpath = midi_folder / file
        if fpath.stat().st_size == 0:
            print(f"Archivo vacío: {file}, se omite.")
            continue

        print("Procesando:", file)
        try:
            midi = converter.parse(fpath)
            parts = instrument.partitionByInstrument(midi)
            elements = parts.parts[0].recurse() if parts else midi.flat.notes
        except Exception as e:
            print(f"Error procesando {file}: {e}")
            continue

        for element in elements:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n.pitch) for n in element.notes))
    return notes


# Extracción por carpeta
notas_clasico = extract_notes(midi_clasico)
notas_videojuegos = extract_notes(midi_videojuegos)

# Combinar notas
notas_combinadas = notas_clasico + notas_videojuegos
print("Total de notas extraídas:", len(notas_combinadas))


Procesando: bach_2.mid
Procesando: bach_1.mid
Procesando: bach_3.mid
Procesando: bach_4.mid
Procesando: bach_0.mid
Procesando: zelda2.mid
Procesando: zelda3.mid
Procesando: super_mario.mid
Procesando: wii.mid
Procesando: zelda1.mid
Total de notas extraídas: 1106


Convertir las notas a algo interpretable (Procesar los datos)

In [11]:
def process_and_save(notes, prefix, output_dir):
    unique_notes = sorted(set(notes))
    note_to_int = {n: i for i, n in enumerate(unique_notes)}
    encoded = [note_to_int[n] for n in notes]

    SEQ_LENGTH = 50
    inputs, targets = [], []
    for i in range(len(encoded) - SEQ_LENGTH):
        inputs.append(encoded[i:i+SEQ_LENGTH])
        targets.append(encoded[i+SEQ_LENGTH])

    inputs = np.array(inputs)
    targets = np.array(targets)

    np.save(output_dir / f"{prefix}_notas.npy", np.array(notes))
    np.savez(output_dir / f"{prefix}_secuencias_datos.npz", inputs=inputs, targets=targets)
    json.dump(unique_notes, open(output_dir / f"{prefix}_notas_unicas.json", "w"))

    print(f"Dataset '{prefix}' guardado: {inputs.shape[0]} secuencias, {len(unique_notes)} notas únicas")

In [12]:
process_and_save(notas_clasico, "clasico", DATA_DIR)
process_and_save(notas_videojuegos, "videojuegos", DATA_DIR)
process_and_save(notas_combinadas, "combinado", DATA_DIR)

Dataset 'clasico' guardado: 0 secuencias, 0 notas únicas
Dataset 'videojuegos' guardado: 1056 secuencias, 178 notas únicas
Dataset 'combinado' guardado: 1056 secuencias, 178 notas únicas


In [13]:
print("\nArchivos generados:")
!ls -lh {DATA_DIR}


Archivos generados:
total 1.1M
-rw-r--r-- 1 root root  128 Nov  5 22:38 clasico_notas.npy
-rw-r--r-- 1 root root    2 Nov  5 22:38 clasico_notas_unicas.json
-rw-r--r-- 1 root root  512 Nov  5 22:38 clasico_secuencias_datos.npz
-rw-r--r-- 1 root root  74K Nov  5 22:38 combinado_notas.npy
-rw-r--r-- 1 root root 1.8K Nov  5 22:38 combinado_notas_unicas.json
-rw-r--r-- 1 root root 422K Nov  5 22:38 combinado_secuencias_datos.npz
drwxr-xr-x 2 root root 4.0K Nov  5 22:36 midi_clasico
drwxr-xr-x 2 root root 4.0K Nov  5 22:36 midi_otro
drwxr-xr-x 2 root root 4.0K Nov  5 22:37 midi_videojuegos
-rw-r--r-- 1 root root  74K Nov  5 22:38 videojuegos_notas.npy
-rw-r--r-- 1 root root 1.8K Nov  5 22:38 videojuegos_notas_unicas.json
-rw-r--r-- 1 root root 422K Nov  5 22:38 videojuegos_secuencias_datos.npz


En este último bloque de código hemos generado 3 archivos importantes por cada género:

**notas.npy**:	Lista completa de las notas extraídas (texto, ej. "C4", "E4.G4")

**secuencias_datos.npz**: Inputs y targets para entrenar la red

**notas_unicas.json**: Diccionario de las notas únicas para volver a convertir nuestras predicciones a notas.

Descargar esto para utilizar en el colab de entrenamiento del modelo: